In [1]:
import os
import random

import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import KFold

from slim_gsgp.datasets.data_loader import *
from preprocessing import get_X_and_y_tensors
from slim_wrapper import SLIMWrapper
from nested_crossval import run_nested_cv_for_model
from utils import *
import pandas as pd
import plotly.express as px
from scipy.stats import wilcoxon
from itertools import combinations

In [2]:
seed = 1111
np.random.seed(seed)
random.seed(seed)

## 1. Getting X an y tensors

In [3]:
X,y=get_X_and_y_tensors()

In [4]:
%load_ext autoreload
%autoreload 2

In [5]:
X

tensor([[ 214.0000, 1725.6000, 1394.0000,  ...,    3.7160,  489.4000,
         2223.3000],
        [ 236.0000, 1769.8000, 1405.4000,  ...,    3.4940,  538.7000,
         2201.8999],
        [ 241.8000, 1724.9000, 1461.7000,  ...,    4.0230,  512.1000,
         2159.8999],
        ...,
        [ 253.1000, 2050.0000, 1669.8000,  ...,    5.1760,  610.0000,
         2648.2000],
        [ 212.1000, 1813.9000, 1468.0000,  ...,    2.8290,  548.4000,
         2262.6001],
        [ 245.7000, 2038.8000, 1655.6000,  ...,    5.9600,  606.9000,
         2704.3000]])

In [6]:
y

tensor([86.1055, 86.1435, 86.4169, 85.9599, 81.6936, 87.5331, 86.8410, 88.2468,
        87.5399, 88.1924, 87.7862, 86.2464, 86.1611, 86.5115, 86.8366, 84.9951,
        86.3812, 88.5102, 85.0129, 85.9784, 88.9662, 84.7507, 87.1134, 87.2657,
        85.5097, 87.1627, 84.8371, 86.8986, 87.4809, 88.2095, 83.4647, 86.3373,
        86.5872, 87.8968, 88.5642, 87.4188, 87.9764, 84.6742, 73.8372, 85.9425,
        86.5554, 87.6478, 87.5996, 86.1728, 88.3539, 87.2642, 86.7197, 85.2699,
        87.7586, 87.8977, 88.1691, 88.4375, 87.1276, 87.5950, 87.9379, 86.5774,
        87.3628, 88.5818, 89.5304, 87.1476, 87.9669, 87.8043, 86.0451, 88.7219,
        88.3917, 89.0096, 85.0285, 87.2786, 87.5549, 87.4504, 88.3307, 89.0777,
        86.8685, 89.4476, 88.6088, 87.6669, 87.6940, 89.0107, 89.6245, 89.4860,
        88.9843, 88.4306, 88.6367, 90.2405, 86.5774, 87.4385, 88.9093, 88.7257,
        88.8421, 88.8315, 87.8210, 88.9991, 88.5073, 90.3756, 91.2113, 88.8250])

## 2. Cross-validation objects

In [7]:
n_outer_cv=30
n_inner_cv=5
outer_cv=KFold(n_splits=n_outer_cv,shuffle=True, random_state=42)

## 3. Applying  SLIM


- `X` will be all the features except the ["CRUDE PROTEIN"], we will try to predict
- `y`will be the ["CRUDE PROTEIN"]
- `fitnesss_function`: rmse
- `minimization`: it will be a minimization problem, we are trying to decrease the rmse

In [8]:
FITNESS_FUNCTION = 'rmse'
MINIMIZATION = True

__`Search space definition`__

- `initializer`: how new random trees are initialized. See [`slim_gsgp` initializers](https://github.com/DALabNOVA/slim/blob/main/slim_gsgp/initializers/initializers.py);
- `tree_constants`: the constants to be used in the terminal set;
- `tree_functions`: the function set (tree internal nodes);
- `prob_const`: the probability for choosing constants instead of dataset features on tree terminals;
- `init_depth`: max depth for tree initialisation;


The following hyperparameter options are the same as for **GP and GSGP**:

- `pop_size`: the size of the population of candidate solutions.
- `elitism`: should the elite(s) be preserved at each generation?
- `n_elits`: if using elitism, how many solutions should be kept?
- Selection method. Only tournament selection in available on `slim_gsgp` libraya, as this is the most commonly used. It requires the definition of the `tournament_size` hyperparameter: how many solutions should participate in the tournament of tournament selection?

The following hyperparameter options are the same as only for **GSGP**:

- `ms_lower`: lower bound for generating the random number used as mutation step.
- `ms_upper`: upper bound for generating the random number used as mutation step.
- `reconstruct`: whether to store the structure of individuals.

**Additionally, SLIM requires:**
- `slim_version`: So far, mutator variation (SLIM+SIG2, SLIM+SIG1, SLIM+ABS, SLIM*SIG2, SLIM*SIG1, SLIM*ABS)
- `p_inflate`: The probability of applying inflate mutation instead of deflate mutation.
- `copy_parent`: Whether to copy the original parent when mutation is impossible (due to depth rules or mutation constraints)

In [9]:
INITIALIZER = 'rhh'
TREE_CONSTANTS = [random.uniform(0, 1) for _ in range(9)]+[ -1.]
TREE_FUNCTIONS = ['add', 'subtract', 'multiply', 'divide']
PROB_CONSTANT = 0.9
MAX_INIT_DEPTH = 4
POP_SIZE = 200
P_XO = 0.9
ELISTISM = True
N_ELITES = 2
TOURNAMENT_SIZE = int(POP_SIZE*0.1)
print(f'TOURNAMENT_SIZE: {TOURNAMENT_SIZE}')

MS_LOWER = 0
MS_UPPER = 1
RECONSTRUCT = True
GENERATIONS = 20
VERBOSE = 1
DATASET_NAME="sustavianfeed"
LOG_LEVEL = 2

SLIM_VERSION = 'SLIM*SIG2'
P_INFLATE = 0.3
COPY_PARENT = False

TOURNAMENT_SIZE: 20


__`Solve settings`__

Same as available for GP and GSGP:

## 4. Hyperparameter tunning with Nested Crossfold validation

We chose nested cross-validation instead of a random train-test split because it gives a fairer and more reliable way to check our model’s performance. It has two steps: one for finding the best model settings (hyperparameters) and another for testing how well the model works on new data. This is important because if we tune and test the model on the same data, the results might look better than they actually are. 
By keeping these steps separate, nested cross-validation helps avoid this problem and gives a more honest estimate of how our model will perform on new, unseen data.

- For the inner fold of the nested cross-validation, we applied Grid Search to take advantage of all its built-in features like systematic parameter tuning and parallel processing. To make this work with the SLIM library, we implemented a custom wrapper called SLIMWrapper, which follows the BaseEstimator interface. This allowed us to integrate the SLIM seamlessly into the grid search process, ensuring efficient and consistent hyperparameter tuning within the inner loop.

In [10]:
estimator=SLIMWrapper(
    init_depth=MAX_INIT_DEPTH,
    tree_constants=TREE_CONSTANTS,
    tree_functions=TREE_FUNCTIONS,
    prob_const = PROB_CONSTANT,
    dataset_name=DATASET_NAME,
    fitness_function=FITNESS_FUNCTION,
    minimization=MINIMIZATION,
    # --
    # SLIM instance 
    pop_size=POP_SIZE,
    initializer=INITIALIZER,
    ms_lower = MS_LOWER,
    ms_upper = MS_UPPER,
    reconstruct = RECONSTRUCT,
    p_inflate=P_INFLATE,
    copy_parent=COPY_PARENT,
    # ---
    # Solve settings
    n_iter=GENERATIONS,
    elitism=ELISTISM,
    n_elites=N_ELITES,
    test_elite=True,
    log_level=LOG_LEVEL,
    verbose=0,
    n_jobs=1,
    seed=seed
)

For this model we shall focus our analysis in 'slim_version'. Our goal is to find which version allows us to have the best performance with all other parameters fixed, for fair comparison, while also trying to make sure our model doesn't grow excessively

In [11]:
param_grid={
    'slim_version': ['SLIM+SIG2', 'SLIM+SIG1', 'SLIM+ABS', 'SLIM*SIG2', 'SLIM*SIG1', 'SLIM*ABS'],
}

In [12]:
summary, general_summary, all_grid_results=run_nested_cv_for_model(X,y, estimator, param_grid, outer_cv, n_inner_cv, model_name="SLIM")


 Outer fold 1

Grid search completed!
Best parameters: {'slim_version': 'SLIM*SIG1'}
Best score: -3.260263965074465
Training with the best hyparameters of 1 fold
Verbose Reporter
-----------------------------------------------------------------------------------------------------------------------------------------
|         Dataset         |  Generation  |     Train Fitness     |       Test Fitness       |        Timing          |      Nodes       |
-----------------------------------------------------------------------------------------------------------------------------------------
|     sustavianfeed       |       0      |   15.269261360168457  |   11.649701118469238     |   0.04793262481689453  |      3           |
|     sustavianfeed       |       1      |   15.269261360168457  |   11.649701118469238     |   0.07361555099487305  |      3           |
|     sustavianfeed       |       2      |   15.269261360168457  |   11.649701118469238     |   0.04355502128601074  |      3     

- Best configuration of parameter per fold

In [13]:
summary.sort_values(by="test_score")

,slim_version,outer_fold,grid_score,test_score
17,SLIM*SIG1,18,-4.391149,tensor(-34.4727)
15,SLIM*SIG1,16,-6.786210,tensor(-7.1474)
6,SLIM*ABS,7,-7.762471,tensor(-5.8096)
2,SLIM*SIG1,3,-5.021016,tensor(-5.5360)
3,SLIM*SIG1,4,-7.234062,tensor(-2.6758)
0,SLIM*SIG1,1,-3.260264,tensor(-2.6046)
7,SLIM*SIG1,8,-3.289378,tensor(-2.1316)
24,SLIM*SIG1,25,-4.426332,tensor(-1.9216)
12,SLIM*SIG1,13,-4.406391,tensor(-1.8386)
20,SLIM*SIG1,21,-5.202611,tensor(-1.7834)


In [14]:
# Log level 2
# -----------
# 0  - Algorithm
# 1  - Instance ID
# 2  - Dataset
# 3  - Seed
# 4  - Generation
# 5  - Fitness
# 6  - Running time
# 7  - Population nodes
# 8  - Test fitness
# 9  - Elite nodes
# 10 - Genotype diversity: gsgp_pop_div_from_vectors (Calculate the diversity of a population from semantic vectors)
# 11 - Phenotype diversity: sd(pop.fit)
# 12 - Log level

fold_1_result=pd.read_csv("./results/SLIM/logs_best_params_fold_1", header=None).head()

In [15]:
fold_1_result.head(20)

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,SLIM+SIG2,2885fe11-35c1-11f0-ab72-c0bfbe084630,sustavianfeed,1111,0,15.209765,0.048923,1186.0,14.483480,3,89475.265625,55490.324000,2
1,SLIM+SIG2,2885fe11-35c1-11f0-ab72-c0bfbe084630,sustavianfeed,1111,1,14.872161,0.106219,3656.0,13.839700,16,127.850334,17.943619,2
2,SLIM+SIG2,2885fe11-35c1-11f0-ab72-c0bfbe084630,sustavianfeed,1111,2,14.732368,0.062658,3314.0,13.559328,27,308.502594,27.946726,2
3,SLIM+SIG2,2885fe11-35c1-11f0-ab72-c0bfbe084630,sustavianfeed,1111,3,14.369542,0.079895,3627.0,12.782021,46,119.292923,10.711985,2
4,SLIM+SIG2,2885fe11-35c1-11f0-ab72-c0bfbe084630,sustavianfeed,1111,4,14.335183,0.055604,3399.0,12.703944,42,4.917006,0.239674,2


In [27]:
plot_fitness_logs('SLIM', n_outer_cv, dataset_name='sustavianfeed')

When looking at folds, we notice that our model tends to stagnate especially in the last 5, 6 epochs with folds showing even bigger stagnation like fold 7 and 18. Even in earlier to mid epochs we notice extended periods of stagnation. In terms of overfitting, the only clear instance happens in fold 16. The model shows learning ability when it is not stagnating but even in the best performing folds we mostly see sharp drops in loss mixed with periods of stagnation.

In [18]:
plot_avg_fitness('SLIM', n_outer_cv, dataset_name='sustavianfeed')

When looking at the average performance we again notice early convergence with the model barely improving after epoch 8.

In [19]:
plot_solution_size_logs('SLIM', n_outer_cv, dataset_name='sustavianfeed')

Most folds have a "staircase" pattern where they intercalate peaks of growth with stagnation. We also notice some folds where the solution decreases in size towards the end (ex: folds 3, 24, 25, 26 e 27).

In [20]:
plot_avg_size("SLIM", n_outer_cv, 'sustavianfeed')

In average, the model shows a pattern of incremental growth without great peaks.

In [21]:
plot_fitness_and_size_logs('SLIM', n_outer_cv, dataset_name='sustavianfeed')

There is a clear tendency for the model to grow albeit not exponentially. The aforementioned periods of stagnation seem to be associated in some folds with periods of noze size stillness. The decreases in node size tend to happen when the model is stuck in performance, meaning that a smaller solution is found to match the current result but not to better it. Bloating is a clear problem since especially between folds 8 and 12, performance tends to stagnate with the node size still increasing.

When considering performance vs size on a per-fold basis, we may conclude that the augment in size is not justified mostly since it is mostly not accompanied by better performance.

In [22]:
RESULTS = './results/SLIM'
CSV = os.path.join(RESULTS, 'all_folds_grid_search_results.csv')
VERSION_COL = 'param_slim_version'
TEST_SCORE_COL = 'mean_test_score'
DATASET = 'sustavianfeed'
LOG_PATTERN = os.path.join(RESULTS, 'logs_best_params_fold_*')


In [23]:
df_csv = pd.read_csv(CSV)
df_csv['test_rmse'] = -df_csv['mean_test_score']
folds_by_version = (
    df_csv
      .groupby('param_slim_version')['outer_fold']
      .unique()
      .apply(list)
      .to_dict()
)
VERSIONS = list(folds_by_version.keys())

In [24]:
#BOXPLOT: TEST RMSE per Version
df_csv = pd.read_csv(CSV)
df_csv['test_rmse'] = -df_csv[TEST_SCORE_COL]  # convert score to RMSE
fig1 = px.box(
    df_csv, x=VERSION_COL, y='test_rmse', points='all',
    title=f"SLIM – Test RMSE Distribution ({DATASET})"
)
fig1.update_layout(width=700, height=400,
                   xaxis_title="SLIM Version", yaxis_title="Test RMSE")
fig1.show()


The first conclusion to be taken is that multiplication provides better results than addition.
The version SLIM * SIG1 appears to be the best for the problem in hand, it has the the lowest bound by far and the distribution of values is closer to it than the upper bound. SLIM * SIG1 has a lower upper bound then every other version so we won't apply any statistical test since our results after plotting are conclusive.

In [25]:
plot_population_fitness_diversity_logs(    
    model_name='SLIM',
    n_folds=n_outer_cv,
    dataset_name='sustavianfeed'
)

Fitness diversity drops almost imeadiatley to close to zero as soon as the model starts iterating. This means that our model is learning too fast, pending heavily towards explotation and neglecting exploration. All solutions are similar meaning the we are facing early convergence. 

In [26]:
plot_population_semantic_diversity_logs(
    model_name='SLIM',
    n_folds=n_outer_cv,
    dataset_name='sustavianfeed'
)

Considering semantic diversity, all solution we observed were similar as we notice very low diversity. This lays credence to our previous conclusion that more focus ought to be put in making our model more exploration focused.

## Conclusions

Overall, the model does not  behave well when compared to node size and with bloating being an clear concern. Clearly the biggest problem is the lack of semantic and fitness diversity. Although the model has achieved good results with RMSE closing in on zero, the fact that the model fails to explore more of the search space might entail the possibility that better solutions were not evaluated.

Given our analysis conclusion that slim version SLIM * SIG1 is the version with better performance, in most cases, for further study we shall focus on improving the model's ability to explore the search space and to prevent early convergence. Overfitting does not constitute a problem as largely the test set follows the tendencies of the train set, the problem is that those tendencies are to mostly to remain static in performance. Given this we don't think that augments the number of generations would be a good policy. Our model needs to be able to explore more so that it doesn't fall into similar solutions that wew will get similar fitnesses and eventually stagnate. We believe that more performance may be gained from exploring a more extensive grid of parameters and more options within those parameters but acknowledge that this would be an approach with diminishing returns due to computational costs.